## Setup

In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import yfinance as yf 

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error 
from statsmodels.tsa.arima_model import ARIMA
plt.style.use('ggplot')

## 1. Data loading 

Outline: Apple stock price data will be obtained from yfinance

Data description: 
- Open: The price at which the stock started trading when the market opened on that day.
- High: The maximum price at which the stock traded during the day.
- Low: The minimum price at which the stock traded during the day.
- Close: The final price at which the stock traded when the market closed on that day.
- Adj Close: The closing price after adjustments for all applicable splits and dividend distributions. This adjusted value is essential for backtesting and analyzing the historical performance of the stock.
- Volume: The total number of shares traded during the trading day. High volume is often associated with high interest in the stock and can indicate significant price movement.

In [3]:
apple_stock = yf.download('AAPL')                               # Downloading all the data associated with apple stock 

[*********************100%%**********************]  1 of 1 completed


In [4]:
apple_stock.shape                                                # (10952, 6) In total 10952 rows and 6 features
apple_stock.index                                                # Index is the datetime index
apple_stock.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1980-12-12,0.128348,0.128906,0.128348,0.128348,0.099058,469033600
1980-12-15,0.122210,0.122210,0.121652,0.121652,0.093890,175884800
1980-12-16,0.113281,0.113281,0.112723,0.112723,0.086998,105728000
1980-12-17,0.115513,0.116071,0.115513,0.115513,0.089152,86441600
1980-12-18,0.118862,0.119420,0.118862,0.118862,0.091737,73449600


## 2. Data cleaning 

### Duplicated values (duplicated rows and columns)

In [5]:
apple_stock.duplicated().sum()                                    # No rows are duplicated 
apple_stock.T.duplicated()                                        # No columns are duplicated 

Open         False
High         False
Low          False
Close        False
Adj Close    False
Volume       False
dtype: bool

### NaN (missing dates and missing rows)

In [6]:
first_date = apple_stock.index.min()
last_date = apple_stock.index.max()
last_date - first_date 

Timedelta('15868 days 00:00:00')

In [7]:
full_range = pd.date_range(start=first_date, end=last_date, freq='D')
full_range.difference(apple_stock.index)                           # This indicates that there are around 4916 days are misiing. 

DatetimeIndex(['1980-12-13', '1980-12-14', '1980-12-20', '1980-12-21',
               '1980-12-25', '1980-12-27', '1980-12-28', '1981-01-01',
               '1981-01-03', '1981-01-04',
               ...
               '2024-04-20', '2024-04-21', '2024-04-27', '2024-04-28',
               '2024-05-04', '2024-05-05', '2024-05-11', '2024-05-12',
               '2024-05-18', '2024-05-19'],
              dtype='datetime64[ns]', length=4916, freq=None)

In [8]:
apple_stock_clean = apple_stock.reindex(full_range)
apple_stock_clean.isna().sum()                                      # Now we have 4916 NaN values which are necessary to be cleaned. 

Open         4916
High         4916
Low          4916
Close        4916
Adj Close    4916
Volume       4916
dtype: int64

I will fill the missing non-trading dates using the last available price (forward fill). This approach assumes that the stock price remained the same during the non-trading period.

In [9]:
apple_stock_clean = apple_stock_clean.fillna(method='ffill')        # forward fill 
apple_stock_clean.isna().sum()                                      # There is no NaN existing in the file. 

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

## 3. Data feature adding -basic

In [10]:
apple_stock_clean.head()

,Open,High,Low,Close,Adj Close,Volume
1980-12-12,0.128348,0.128906,0.128348,0.128348,0.099058,469033600.0
1980-12-13,0.128348,0.128906,0.128348,0.128348,0.099058,469033600.0
1980-12-14,0.128348,0.128906,0.128348,0.128348,0.099058,469033600.0
1980-12-15,0.122210,0.122210,0.121652,0.121652,0.093890,175884800.0
1980-12-16,0.113281,0.113281,0.112723,0.112723,0.086998,105728000.0


In [12]:
# We want to add a return column which indicate the change of the stock price 
apple_stock_clean['Return'] = apple_stock_clean['Adj Close'].pct_change()       # In this case the first row will be the NAN value that we need to drop. 
apple_stock_clean.dropna(inplace=True)
apple_stock_clean

,Open,High,Low,Close,Adj Close,Volume,Return
1980-12-13,0.128348,0.128906,0.128348,0.128348,0.099058,469033600.0,0.000000
1980-12-14,0.128348,0.128906,0.128348,0.128348,0.099058,469033600.0,0.000000
1980-12-15,0.122210,0.122210,0.121652,0.121652,0.093890,175884800.0,-0.052171
1980-12-16,0.113281,0.113281,0.112723,0.112723,0.086998,105728000.0,-0.073398
1980-12-17,0.115513,0.116071,0.115513,0.115513,0.089152,86441600.0,0.024751
...,...,...,...,...,...,...,...
2024-05-19,189.509995,190.809998,189.179993,189.869995,189.869995,41282900.0,0.000000
2024-05-20,189.330002,191.919998,189.009995,191.039993,191.039993,44361300.0,0.006162
2024-05-21,191.089996,192.729996,190.919998,192.350006,192.350006,42309400.0,0.006857
2024-05-22,192.270004,192.820007,190.270004,190.899994,190.899994,34648500.0,-0.007538


In [14]:
# Save the cleaned data file in the Data folder
apple_stock_clean.to_csv(r'C:\Users\12436\Desktop\BrainStation\Capstone project\capstone-Leoyuyuyu\data\apple_stock_clean.csv')